### 환경 설정

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from openai import OpenAI

### 1. 사용자의 의도 분류 하기

In [4]:
from string import Template

query = "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"

system_prompt = """
너는 항공편 관련 상담을 해주는 인공지능 에이전트야.
항공편 관련 상담에는 다음과 같은 사용자의 요청이 있을 수 있어.

항공편 예약: 사용자가 특정 날짜와 시간에 항공편을 예약하려고 할 때 해당 요청을 처리합니다.
항공편 변경 또는 취소: 이미 예약된 항공편을 변경하거나 취소하고자 할 때 이에 대한 요청을 처리합니다.
좌석 선택: 특정 좌석을 선택하거나 선호하는 좌석을 예약할 수 있도록 도와주는 요청을 처리합니다.
가격 및 할인 문의: 항공편의 가격 및 할인 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
수하물 정보: 수하물의 허용량, 추가 비용, 수하물 체크인 절차 등에 대한 정보를 제공하는 요청을 처리합니다.
항공편 상태 확인: 특정 항공편의 운항 상태, 지연 여부, 취소 여부 등에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
탑승 시 요구되는 문서: 여권, 비자 등 탑승 시 필요한 문서에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
환불 및 보상 요청: 항공편 취소 시 환불 절차, 보상 요청 등에 대한 요청을 처리합니다.
장애물 도움: 장애인 또는 특별한 요구사항이 있는 승객을 위한 보조 서비스 요청을 처리합니다.
기타 문의: 항공편 관련 기타 문의 사항에 대한 답변을 제공합니다.
"""

prompt_template = Template("""
다음 사용자의 요청에서 사용자의 의도를 분류해줘
질문: "$query"
사용자 의도:
""")

print(system_prompt)
print(prompt_template.substitute(query=query))


너는 항공편 관련 상담을 해주는 인공지능 에이전트야.
항공편 관련 상담에는 다음과 같은 사용자의 요청이 있을 수 있어.

항공편 예약: 사용자가 특정 날짜와 시간에 항공편을 예약하려고 할 때 해당 요청을 처리합니다.
항공편 변경 또는 취소: 이미 예약된 항공편을 변경하거나 취소하고자 할 때 이에 대한 요청을 처리합니다.
좌석 선택: 특정 좌석을 선택하거나 선호하는 좌석을 예약할 수 있도록 도와주는 요청을 처리합니다.
가격 및 할인 문의: 항공편의 가격 및 할인 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
수하물 정보: 수하물의 허용량, 추가 비용, 수하물 체크인 절차 등에 대한 정보를 제공하는 요청을 처리합니다.
항공편 상태 확인: 특정 항공편의 운항 상태, 지연 여부, 취소 여부 등에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
탑승 시 요구되는 문서: 여권, 비자 등 탑승 시 필요한 문서에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
환불 및 보상 요청: 항공편 취소 시 환불 절차, 보상 요청 등에 대한 요청을 처리합니다.
장애물 도움: 장애인 또는 특별한 요구사항이 있는 승객을 위한 보조 서비스 요청을 처리합니다.
기타 문의: 항공편 관련 기타 문의 사항에 대한 답변을 제공합니다.


다음 사용자의 요청에서 사용자의 의도를 분류해줘
질문: "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"
사용자 의도:



In [5]:
client = OpenAI()


def consult(query):
  prompt = prompt_template.substitute(query=query)
  chat_completion = client.chat.completions.create(
      messages=[
          {
            "role": "system",
            "content": system_prompt,
          },
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-4o-mini",
  )
  return chat_completion.choices[0].message.content

consult("안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?")

'사용자의 의도는 "항공편 예약"입니다. 사용자가 오늘 날짜로 뉴욕행 비행기를 예약하고 싶어하는 요청입니다.'

### 2. 필요 정보 추출(엔티티 추출)

In [6]:
from string import Template

query = "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"

system_prompt = """
너는 항공편 관련 상담을 해주는 인공지능 에이전트야.
항공편 관련 상담에는 다음과 같은 사용자의 요청이 있을 수 있어.

항공편 예약: 사용자가 특정 날짜와 시간에 항공편을 예약하려고 할 때 해당 요청을 처리합니다.
항공편 변경 또는 취소: 이미 예약된 항공편을 변경하거나 취소하고자 할 때 이에 대한 요청을 처리합니다.
좌석 선택: 특정 좌석을 선택하거나 선호하는 좌석을 예약할 수 있도록 도와주는 요청을 처리합니다.
가격 및 할인 문의: 항공편의 가격 및 할인 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
수하물 정보: 수하물의 허용량, 추가 비용, 수하물 체크인 절차 등에 대한 정보를 제공하는 요청을 처리합니다.
항공편 상태 확인: 특정 항공편의 운항 상태, 지연 여부, 취소 여부 등에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
탑승 시 요구되는 문서: 여권, 비자 등 탑승 시 필요한 문서에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
환불 및 보상 요청: 항공편 취소 시 환불 절차, 보상 요청 등에 대한 요청을 처리합니다.
장애물 도움: 장애인 또는 특별한 요구사항이 있는 승객을 위한 보조 서비스 요청을 처리합니다.
기타 문의: 항공편 관련 기타 문의 사항에 대한 답변을 제공합니다.

또한 항공편 관련 상담에서 구분해야 하는 항공편 관련 키워드는 다음과 같아
- 출발지 공항 또는 도시
- 도착지 공항 또는 도시
- 출발 일자와 시간
- 도착 일자와 시간
- 여행자 인원 (성인, 어린이, 유아 수)
- 선호하는 항공사 (선택사항)
- 좌석 등급 (일반석, 비즈니스석, 일등석 등)
- 예약자 정보 (이름, 연락처, 이메일 주소)
- 추가 서비스나 특별 요청 (식사 요청, 보조 서비스 등)
- 가격 및 할인 정보
"""

prompt_template = Template("""
다음 사용자의 요청에서 사용자의 의도와 주요 키워드를 분류해줘

질문: "4월 3일에 런던에 도착하는 비행기에서 식사를 제공받을 수 있나요?"
사용자의 의도: '기타 문의'
키워드: {'추가 서비스나 특별 요청': '식사 제공', '도착지 공항 또는 도시': '런던', '도착 일자와 시간': '4월 3일'}

질문: "내일 뉴욕행 비행기 좌석을 변경하고 싶은데 가능할까요?"
사용자의 의도: '항공편 변경 또는 취소'
키워드: {'출발지 공항 또는 도시': '뉴욕', '좌석 등급': '변경'}

질문: "5월 15일 오전 10시에 파리로 가는 항공편을 예약하고 싶습니다."
사용자의 의도: '항공편 예약'
키워드: {'출발 일자와 시간': '5월 15일 오전 10시', '도착지 공항 또는 도시': '파리'}

질문: "내일 도착 예정인 터키행 항공편이 지연되었는지 확인해주세요."
사용자의 의도: '항공편 상태 확인'
키워드: {'도착지 공항 또는 도시': '터키', '도착 일자와 시간': '내일'}

질문: "항공편 가격이 할인 중인가요? 현재 할인 정보를 알려주세요."
사용자의 의도: '가격 및 할인 문의'
키워드: {'가격 및 할인 정보': '할인'}

질문: "$query":
""")

print(system_prompt)
print(prompt_template.substitute(query=query))


너는 항공편 관련 상담을 해주는 인공지능 에이전트야.
항공편 관련 상담에는 다음과 같은 사용자의 요청이 있을 수 있어.

항공편 예약: 사용자가 특정 날짜와 시간에 항공편을 예약하려고 할 때 해당 요청을 처리합니다.
항공편 변경 또는 취소: 이미 예약된 항공편을 변경하거나 취소하고자 할 때 이에 대한 요청을 처리합니다.
좌석 선택: 특정 좌석을 선택하거나 선호하는 좌석을 예약할 수 있도록 도와주는 요청을 처리합니다.
가격 및 할인 문의: 항공편의 가격 및 할인 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
수하물 정보: 수하물의 허용량, 추가 비용, 수하물 체크인 절차 등에 대한 정보를 제공하는 요청을 처리합니다.
항공편 상태 확인: 특정 항공편의 운항 상태, 지연 여부, 취소 여부 등에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
탑승 시 요구되는 문서: 여권, 비자 등 탑승 시 필요한 문서에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
환불 및 보상 요청: 항공편 취소 시 환불 절차, 보상 요청 등에 대한 요청을 처리합니다.
장애물 도움: 장애인 또는 특별한 요구사항이 있는 승객을 위한 보조 서비스 요청을 처리합니다.
기타 문의: 항공편 관련 기타 문의 사항에 대한 답변을 제공합니다.

또한 항공편 관련 상담에서 구분해야 하는 항공편 관련 키워드는 다음과 같아
- 출발지 공항 또는 도시
- 도착지 공항 또는 도시
- 출발 일자와 시간
- 도착 일자와 시간
- 여행자 인원 (성인, 어린이, 유아 수)
- 선호하는 항공사 (선택사항)
- 좌석 등급 (일반석, 비즈니스석, 일등석 등)
- 예약자 정보 (이름, 연락처, 이메일 주소)
- 추가 서비스나 특별 요청 (식사 요청, 보조 서비스 등)
- 가격 및 할인 정보


다음 사용자의 요청에서 사용자의 의도와 주요 키워드를 분류해줘

질문: "4월 3일에 런던에 도착하는 비행기에서 식사를 제공받을 수 있나요?"
사용자의 의도: '기타 문의'
키워드: {'추가 서비스나

In [7]:
def consult2(query):
  prompt = prompt_template.substitute(query=query)
  chat_completion = client.chat.completions.create(
      messages=[
          {
            "role": "system",
            "content": system_prompt,
          },
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-3.5-turbo",
  )
  return chat_completion.choices[0].message.content


consult2("안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?")

"사용자의 의도: '항공편 예약'\n키워드: {'출발지 공항 또는 도시': '뉴욕', '출발 일자와 시간': '오늘 날짜'}"

### 3. 대화 관리

In [9]:
hist = ""
class ConsultBot():
    def __init__(self):
        self.hist = ""
        self.system_prompt = """
        너는 항공편 관련 상담을 해주는 인공지능 에이전트야.
        아래 정보를 바탕으로 사용자에서 항공편 예약 관련 상담을 친절하게 진행해 주어야 해

        항공편 예약을 하기 위해 참고해야 하는 정보는 다음과 같아.

        - 출발지 공항 또는 도시
        - 도착지 공항 또는 도시
        - 출발 일자와 시간
        - 도착 일자와 시간
        - 여행자 인원 (성인, 어린이, 유아 수)
        - 선호하는 항공사 (선택사항)
        - 좌석 등급 (일반석, 비즈니스석, 일등석 등)
        - 예약자 정보 (이름, 연락처, 이메일 주소)
        - 추가 서비스나 특별 요청 (식사 요청, 보조 서비스 등)
        - 가격 및 할인 정보


        """
        self.prompt_template = Template("""
            사용자와 항공편 예약 상담을 진행해줘.

            이전 대화: $history
            질문: "$query":
        """
        )

    def consult(self, query):
        prompt = self.prompt_template.substitute(history=self.hist, query=query)
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="gpt-3.5-turbo",
        )
        answer = chat_completion.choices[0].message.content
        self.hist += "질문: " + query + "\n답변: " + answer
        return answer


consult_bot = ConsultBot()
consult_bot.consult("안녕하세요. 인천 공항에서 출발하는 뉴욕행 비행기를 예약할 수 있나요?")

'안녕하세요! 네, 인천 공항에서 출발하는 뉴욕행 비행기를 예약해드릴 수 있어요. 몇 분의 여행자가 이동하시나요? 그리고 선호하시는 출발 일자와 도착 일자를 알려주세요. 선호하시는 항공사가 있으시거나 좌석 등급을 원하신다면 함께 말씀해주세요. 부가 서비스나 특별 요청사항이 있으실 경우도 알려주세요. 부탁드려도 되나요?'

In [10]:
print(consult_bot.consult("4월 3일에 출발하는 비행기를 예약하고 싶어요, 비즈니스 석으로 2명 예약해 주세요"))

네, 4월 3일에 인천 공항에서 뉴욕행 비즈니스석으로 2명을 예약해 드리겠습니다. 추가로 필요한 정보를 몇 가지 더 물어봐도 될까요?출발 시간이나 도착 시간, 좌석 선호도, 예약자 정보 등이 필요할 수 있습니다.
